In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange
from sklearn import datasets
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from MegaSAM.data.moons import moons_data_gen
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
N = 1000

all_data, train_loader, test_loader, test_data = moons_data_gen(N=1000, batch_size=1000)


In [3]:
output1 = 10

model = nn.Sequential(
    nn.Linear(2, output1),
    nn.ReLU(),
    nn.Linear(output1, 1)
)

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

num_params = get_n_params(model)
M = torch.ones(num_params)

epochs = 1000

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

for epoch in trange(epochs):
    for points, labels in train_loader:
        
        optimizer.zero_grad()

        x = points[:,:,0].float()
        x = x.to(device)
        labels = labels[:,None].double()


        y = model(x)
        loss = criterion(y, labels)
        
        loss.backward()
        optimizer.step()

correct_test = 0
with torch.no_grad():
        # Iterate through test set minibatchs 
        for numbers2, labels2 in test_loader:
            numbers2 = numbers2
            labels2 = labels2.double()[:,None]
            # Forward pass
            x2 = numbers2[:,:,0].float()
            y2 = model(x2)
            predictions2 = ((y2>0)*1)
            # print(predictions2)
            # print(labels2)
            correct_test += torch.sum((predictions2 == labels2).float())
            # print(correct_test)
print(f'Test accuracy: {correct_test/len(test_data)}')


  0%|          | 0/1000 [00:00<?, ?it/s]

Test accuracy: 0.8650000095367432
